In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle

Generate and Save Input Data - Only run once.

In [ ]:
target_dt = 1.0/30 #30 frames per second?

#load combined file of annotated child-child data from PInSoRo
data = pd.read_csv("multidata.csv", low_memory=False) 
#array of data for purple child (points in space for each frame)
x = np.array(data.iloc[:,11:195]).astype(float) 
#array of labels (purple child annotations, engagement) 
labs = np.array(data.iloc[:,443]).astype(str) 
a = []
#for each column in the array, fill in any blanks by interpolating between available values
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    if i == 180:               
        y[y>0]-=np.pi*2   
        y += np.pi            
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T

#function to extract consecutive frames with the same label (i.e. clips)
def extract_pattern(start, end, target_dt): 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)
    timestamp = np.array(data.iloc[start:end,0]).astype(str)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    #t = np.arange(int(t_sample[-1]/target_dt))*target_dt
    t = np.arange(end-start)*target_dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return timestamp, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])

t_high=[]
p_high=[]
t_mid=[]
p_mid=[]
t_low=[]
p_low=[]

for i in range(1,(len(start)-1)):
    try:
        if label[i]==('goaloriented'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_high.append(ts)
            p_high.append(pi)
        if label[i]==('aimless'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_mid.append(ts)
            p_mid.append(pi)
        if label[i]==('noplay'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_low.append(ts)
            p_low.append(pi)
    except IndexError:
        print('empty pattern')
        
i=0
high = []
mid = []
low = []
for entry in t_high:
    high.append([t_high[i],p_high[i]])
    i = i+1
i=0
for entry in t_mid:
    mid.append([t_mid[i],p_mid[i]])
    i = i+1
i=0
for entry in t_low:
    low.append([t_low[i],p_low[i]])
    i = i+1
    
np.save('higheng.npy', high)
np.save('mideng.npy', mid)
np.save('loweng.npy', low)

Set up network.

In [ ]:
class PatternInterpolationTrain(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        self.param('length of training pattern', len_train=10),
        self.param('number of epochs', n_epoch=1),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        high = list(np.load('higheng.npy'))
        mid = list(np.load('mideng.npy'))
        low = list(np.load('loweng.npy'))
        
        seed=param.p_seed

        dt = 0.001
        target_dt = 1.0/30 #data sampled at 30Hz
        D = param.n_dims
        classify_score = {}
        accuracy = {}
        
        #shuffle data
        p_high2 = random.sample(high, len(high))
        p_low2 = random.sample(low, len(low))
        p_mid2 = random.sample(mid, len(mid))
        p_mid2 = np.array(p_mid2)
        
        #Create 80% training sets
        high_train_ts = np.array(p_high2[:(int(len(p_high2)*0.8))]) 
        high_train_list=list(high_train_ts[:,1])
        high_train_time=list(high_train_ts[:,0])
        
        low_train_ts = np.array(p_low2[:(int(len(p_low2)*0.8))]) 
        low_train_list=list(low_train_ts[:,1])
        low_train_time=list(low_train_ts[:,0])
        
        #Create 20% testing sets
        high_test_ts = np.array(p_high2[(int(len(p_high2)*0.8)):]) 
        high_test_list=list(high_test_ts[:,1])
        high_test_time=list(high_test_ts[:,0])

        low_test_ts = np.array(p_low2[(int(len(p_low2)*0.8)):]) 
        low_test_list=list(low_test_ts[:,1])
        low_test_time=list(low_test_ts[:,0])
        
        #Remove empty patterns (only found in low engagement set)
        for p in low_train_list:
            if p.size > 0:
                pass
            else:
                low_train_list.remove(p)

        for p in low_test_list:
            if p.size >0:
                pass
            else:
                low_test_list.remove(p)

        #Create PCA model using training data and save model
        train_all = np.vstack(high_train_list+low_train_list)
        pca_model = PCA(n_components=D).fit(train_all)
        
        pickle_filename = "%s_pca_model.pkl" % seed
        with open(pickle_filename, 'wb') as file:
            pickle.dump(pca_model, file)

        #Save timestamps for unstacking data later
        train_time_all = np.hstack(high_train_time+low_train_time)
        test_time_all = np.hstack(high_test_time+low_test_time)
        
        np.save('%s_high_train_time.npy' % seed, high_train_time)
        np.save('%s_low_train_time.npy' % seed, low_train_time)

########## TRANSFORM TRAINING PATTERNS INTO PCA COMPONENTS #########
        high_train_pca = np.vstack([pca_model.transform(p) for p in high_train_list])
        low_train_pca = np.vstack([pca_model.transform(p) for p in low_train_list])

        np.save('%s_hightrain_pca.npy' % seed, high_train_pca)
        np.save('%s_low_train_pca.npy' % seed, low_train_pca)
        
########## TRANSFORM TESTING PATTERNS - ONE CLIP AT A TIME #########
        high_test_pca2=[]
        low_test_pca2=[]
        mid_test_pca2=[]

        for i in range(len(high_test_list)):
            high_test_pca2.append(pca_model.transform(high_test_list[i]))

        for i in range(len(low_test_list)):
            low_test_pca2.append(pca_model.transform(low_test_list[i]))

            
        mid_list=list(p_mid2[:,1])
        mid_time=list(p_mid2[:,0])
        for i in range(len(mid_list)):
            mid_test_pca2.append(pca_model.transform(mid_list[i]))
            
        #Save testing data
        np.save('%s_high_test_pca2.npy' % seed, high_test_pca2)
        np.save('%s_low_test_pca2.npy' % seed, low_test_pca2)
        np.save('%s_mid_test_pca2.npy' % seed, mid_test_pca2)
        
        #Save timestamps
        np.save('%s_high_test_time.npy' % seed, high_test_time)
        np.save('%s_low_test_time.npy' % seed, low_test_time)
        np.save('%s_mid_time.npy' % seed, mid_time)
#############################################################

        T_train = param.len_train   # number of seconds to train on for each class
        T_test = 100    # number of seconds to test on for each class
        
        high_train_time2 = [item for sublist in high_train_time for item in sublist]
        low_train_time2 = [item for sublist in low_train_time for item in sublist]

        N_frames = int(T_train*30)
        train_timestamps = np.hstack([high_train_time2[:(N_frames*param.n_epoch)], 
                                      low_train_time2[:(N_frames*param.n_epoch)]])
        
############ TRAINING WITH 80% ############ 
        print('Run Number: ', int(param.p_seed)+1)
        print('Testing with 80% Training Data')
    
        num_epochs = param.n_epoch
        batch_size = int(T_train*30)
        batch = 0
        theta = 15

        for epoch in range(num_epochs):
            training = np.vstack([high_train_pca[batch:int(batch+batch_size)], 
                                  low_train_pca[batch:int(batch+batch_size)]])
            batch += batch_size

            training_net = nengo.Network(seed=seed)#param.seed)
            with training_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([high_train_pca[:,i], low_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))


                node_pool = nengo.Node(None, size_in=rw[0].state.size_out*D)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, node_pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(training, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_node_pool = nengo.Probe(node_pool, sample_every = 0.1)


            sim = nengo.Simulator(training_net)
            with sim:
                sim.run(T_train*2)  


            pool_model = nengo.Network()
            with pool_model:
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)
            pool_sim = nengo.Simulator(pool_model)

            import nengo.utils.ensemble

            _, a = nengo.utils.ensemble.tuning_curves(pool, pool_sim, inputs=sim.data[p_node_pool])

            if epoch == 0:
                a_high = a[:(int(len(a)/2))]
                a_low = a[(int(len(a)/2)):]
            else:
                a_high = np.concatenate((a_high, a[:(int(len(a)/2))]))
                a_low = np.concatenate((a_low, a[(int(len(a)/2)):]))

        a_out = np.vstack([a_high, a_low])


        N = int((len(a_out))/2) #int(T_train*1000)
        target = np.hstack([np.ones(N), -np.ones(N)]).reshape(-1, 1)
        dec, info = nengo.solvers.LstsqL2(reg=0.1)(a_out, target)
        
        np.save('%s_decoder.npy' % seed, dec)
        np.save('%s_node_pool.npy' % seed, node_pool.size_out)
        
        v = np.dot(a_out, dec)


        train_timestamps = train_timestamps[0::2]

        output_high_train = {"output_high_train":{"time":train_timestamps[:N], "output":v[:N]}}
            
        output_low_train = {"output_low_train":{"time":train_timestamps[N:], "output":v[N:]}}

############ SAVE DATA ############ 
       
        results = {**output_high_train, **output_low_train}

        return results            


Run network.

In [ ]:
for seed in range(20):
    PatternInterpolationTrain().run(seed=seed, 
                                    p_seed=seed, 
                                    len_train=500, 
                                    n_epoch=3, 
                                    data_dir='Training_Out', 
                                    data_format="npz")